### INFO for functions

The "fit_and_score_features" function plays a foundational role in the feature selection process by calculating scores for each feature in dataset X, utilizing the Cox Proportional Hazards model. These scores are indicative of each feature's association with the survival outcome, represented by dataset y. This function, alongside "remove_high_correlated_features" and "feature_selection", establishes a robust framework for identifying the most significant features in a dataset. The "remove_high_correlated_features" function further refines the feature set by eliminating features with high correlation. Subsequently, "feature_selection" utilizes these functions to determine the optimal subset of features using a combination of Cox model scoring and GridSearchCV.

Building upon this foundation, the functions "CV_on_training_data", "result_on_testing_data", and "CV_on_all_dataset" extend the model's application by implementing various forms of cross-validation and testing with GradientBoostingSurvivalAnalysis. These functions are crucial for evaluating the model's performance, fitting models, making predictions, and calculating the Concordance index. They systematically use the dataset (X, Y), the selected features, and other pertinent parameters to execute necessary evaluations.

In [ ]:
# General Imports 
import numpy as np
import pandas as pd 
from sklearn.model_selection import GridSearchCV, KFold
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sklearn.preprocessing import StandardScaler
import sksurv
from sklearn.pipeline import Pipeline
import warnings
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

In [ ]:
def fit_and_score_features(X, y):
    """
    Calculate scores for each feature in X based on its association with the survival outcome y using Cox Proportional Hazards model.
    
    Parameters:
    X (DataFrame): Feature data.
    y (structured array): Survival outcome data with fields 'time' and 'event'.
    
    Returns:
    scores (ndarray): Array of scores for each feature.
    """
    # Convert the feature dataframe to a numpy array for processing
    X = np.array(X)
    
    # Determine the number of features
    n_features = X.shape[1]
    
    # Initialize an array to hold the score for each feature
    scores = np.empty(n_features)
    
    # Initialize the Cox Proportional Hazards model
    m = CoxPHSurvivalAnalysis()
    
    # Iterate over each feature to calculate its score
    for j in range(n_features):
        # Extract the feature column
        Xj = X[:, j:j+1]
        
        # Fit the Cox model to the feature and survival data
        m.fit(Xj, y)
        
        # Store the score (e.g., concordance index) of the feature
        scores[j] = m.score(Xj, y)
    
    # Return the array of scores, one for each feature
    return scores   

In [ ]:
def remove_high_correlated_features(_X_, _VALUE_): 
    """
    Remove features from X that have a correlation higher than VALUE using Spearman rank correlation.
    
    Parameters:
    _X_ (DataFrame): Feature data.
    _VALUE_ (float): Threshold for removing correlated features.
    
    Returns:
    _X_ (DataFrame): Feature data with highly correlated features removed.
    """
    # Calculate the Spearman correlation matrix and take the absolute value of correlations
    corr_matrix = _X_.corr(method ='spearman').abs()
    
    # Find the upper triangle of the correlation matrix to avoid duplication
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Identify columns that have any correlation value above the threshold _VALUE_
    to_drop = [column for column in upper.columns if any(upper[column] >= _VALUE_)]
    
    # Drop the identified columns from the dataset
    _X_ = _X_.drop(to_drop, axis=1)
    return _X_

In [ ]:
def feature_selection(_X_, _Y_):
    """
    Select best features from X based on their association with survival outcome Y using Cox model and GridSearch.
    
    Parameters:
    _X_ (DataFrame): Feature data.
    _Y_ (structured array): Survival outcome data.
    
    Returns:
    selected_features (list): List of selected feature names.
    """    
    # Calculate scores for each feature using the Cox Proportional Hazards model
    scores = fit_and_score_features(_X_, _Y_)
    
    # Sort the features based on their scores in descending order
    a = pd.Series(scores, index=_X_.columns).sort_values(ascending=False)
    
    # Setup a pipeline with feature selection and Cox model
    pipe = Pipeline([('select', SelectKBest(fit_and_score_features, k=3)), ('model', CoxPHSurvivalAnalysis())])
    
    # Ignore warnings during GridSearch
    warnings.filterwarnings("ignore")
    
    # Define the grid search parameters
    param_grid = {'select__k': np.arange(1, 10)}
    cv = KFold(n_splits=3, random_state=1, shuffle=True)
    
    # Perform GridSearchCV
    gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv)
    
    # Standardize the feature data
    SS = StandardScaler().fit(_X_)
    train_x_SS = SS.transform(_X_)
    
    # Fit the GridSearchCV pipeline
    gcv.fit(train_x_SS, _Y_)
    
    # Get the GridSearchCV results and select the best features
    results = pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)
    results.loc[:, ~results.columns.str.endswith("_time")]
    selected_features = list(a.index[0:list(gcv.best_params_.values())[0]])
    return selected_features

In [ ]:
def CV_on_training_data(_X_, _Y_, _SELECTED_FEATURES_, _FOLDS_):    
    """
    Perform cross-validation on training data using Gradient Boosting for survival analysis.
    
    Parameters:
    _X_ (DataFrame): Feature data for training.
    _Y_ (structured array): Survival outcome data for training.
    _SELECTED_FEATURES_ (list): List of selected feature names.
    _FOLDS_ (int): Number of folds for cross-validation.
    
    Returns:
    results_list (list): List of C-index scores for each fold.
    """
    # Initialize K-Fold cross-validation
    kf = KFold(n_splits = _FOLDS_, shuffle = True, random_state = 0)
    a = range(0,len(_X_))
    k_fold= [x for x in kf.split(a)]
    
    results_list = []
    for i in range(0,_FOLDS_):
        # Select training and validation data for the current fold
        train_x_selected = (_X_.iloc[k_fold[i][0]])[_SELECTED_FEATURES_].values
        validation_x_selected = (_X_.iloc[k_fold[i][1]])[_SELECTED_FEATURES_].values
        y_train = _Y_[k_fold[i][0]]
        y_validation = _Y_[k_fold[i][1]]
        
        # Initialize the estimator with predefined parameters
        estimator = GradientBoostingSurvivalAnalysis(n_estimators= 100,learning_rate = 0.001, max_depth = 4, min_samples_split=10,min_samples_leaf=2, max_features=1, random_state=0)

        # Standardize the features
        SS = StandardScaler().fit(train_x_selected)
        train_x_selected = SS.transform(train_x_selected)
        validation_x_selected = SS.transform(validation_x_selected)
        
        # Fit the model and calculate the C-index for the current fold
        estimator.fit(train_x_selected, y_train)
        result = estimator.score(validation_x_selected, y_validation)
        results_list.append(result)
    
    # Output the mean and standard deviation of C-index scores
    print(f"C-index mean:", np.mean(results_list))
    print(f"C-index standard deviation:", np.std(results_list))
    return results_list

In [ ]:
def result_on_testing_data(_X_training_, _Y_training_, _X_testing_, _Y_testing_, _SELECTED_FEATURES_):
    """
    Evaluate the model on testing data and return the C-index score, predictions, estimator, and transformed testing features.
    
    Parameters:
    _X_training_ (DataFrame): Feature data for training.
    _Y_training_ (structured array): Survival outcome data for training.
    _X_testing_ (DataFrame): Feature data for testing.
    _Y_testing_ (structured array): Survival outcome data for testing.
    _SELECTED_FEATURES_ (list): List of selected feature names.
    
    Returns:
    Tuple containing the C-index score, predictions, fitted estimator, and transformed testing features.
    """
    
    # Initialize the estimator with predefined parameters
    estimator = GradientBoostingSurvivalAnalysis(n_estimators= 100,learning_rate = 0.001, max_depth = 4, min_samples_split=10,min_samples_leaf=2, max_features=1, random_state=0)

    # Standardize the training features and transform the testing features
    SS = StandardScaler().fit(_X_training_[_SELECTED_FEATURES_])
    train_x_selected = SS.transform(_X_training_[_SELECTED_FEATURES_])
    test_x_selected = SS.transform(_X_testing_[_SELECTED_FEATURES_])
    
    # Fit the model on training data and evaluate on testing data
    estimator.fit(train_x_selected, _Y_training_)
    print(estimator.score(test_x_selected, _Y_testing_))
    return estimator.score(test_x_selected, _Y_testing_), estimator.predict(test_x_selected), estimator, test_x_selected

In [ ]:
def CV_on_all_datasat(_X_, _Y_, _SELECTED_FEATURES_, _FOLDS_):
    """
    Perform cross-validation on the entire dataset using Gradient Boosting for survival analysis and return the C-index results.

    Parameters:
    _X_ (DataFrame): The entire feature dataset.
    _Y_ (structured array): The entire survival outcome dataset.
    _SELECTED_FEATURES_ (list): List of feature names selected for the model.
    _FOLDS_ (int): The number of folds to use in cross-validation.

    Returns:
    C_index_results_list (list): List of C-index results from each fold.
    cv_dataframe (DataFrame): Compiled results from all folds including predicted risk scores and selected features.
    """    
    # Initialize KFold cross-validation
    kf = KFold(n_splits = _FOLDS_, shuffle = True, random_state = 0)
    
    # Generate indices for splitting data
    a = range(0,len(_X_))
    k_fold= [x for x in kf.split(a)]
    
    # Initialize list to store C-index results
    C_index_results_list = []    
    
    # Prepare dataframe to store cross-validation results
    name_columns = list(_X_[_SELECTED_FEATURES_].columns) + list(_Y_.dtype.names) + ['First_TYPE_Treatment']
    cv_dataframe = pd.DataFrame(columns = name_columns)
    
    # Loop through each fold for cross-validation
    for i in range(0,_FOLDS_):
        # Select training and validation data for the current fold
        train_x_selected = (_X_.iloc[k_fold[i][0]])[_SELECTED_FEATURES_ + ['First_TYPE_Treatment']]
        train_First_TYPE_Treatment = train_x_selected[['First_TYPE_Treatment']]
        train_x_selected = train_x_selected[_SELECTED_FEATURES_]        
        validation_x_selected = (_X_.iloc[k_fold[i][1]])[_SELECTED_FEATURES_ + ['First_TYPE_Treatment']]
        validation_First_TYPE_Treatment = validation_x_selected[['First_TYPE_Treatment']]
        validation_x_selected = validation_x_selected[_SELECTED_FEATURES_]        
        y_train = _Y_[k_fold[i][0]]
        y_validation = _Y_[k_fold[i][1]]
        
        # Initialize the Gradient Boosting Survival Analysis estimator
        estimator = GradientBoostingSurvivalAnalysis(n_estimators= 100,learning_rate = 0.001, max_depth = 4, min_samples_split=10,min_samples_leaf=2, max_features=1, random_state=0)
        
        # Standardize the features
        SS = StandardScaler().fit(train_x_selected)
        temp = SS.transform(train_x_selected)
        train_x_selected = pd.DataFrame(temp, index = train_x_selected.index, columns = train_x_selected.columns)                
        temp = SS.transform(validation_x_selected)
        validation_x_selected = pd.DataFrame(temp, index = validation_x_selected.index, columns = validation_x_selected.columns)   
        
        # Fit the model and compute the C-index on the validation set
        estimator.fit(train_x_selected, y_train)
        result = estimator.score(validation_x_selected, y_validation)
        C_index_results_list.append(result)        
        
        # Store the validation results and risk scores in a dataframe
        temp_dataframe = pd.concat([validation_x_selected, pd.DataFrame(y_validation, index = validation_x_selected.index, columns = y_validation.dtype.names)], axis = 1)
        temp_dataframe['Risk_Score'] = estimator.predict(validation_x_selected)
        temp_dataframe = pd.merge(temp_dataframe, validation_First_TYPE_Treatment, on="ID", how="inner")
        cv_dataframe = pd.concat([cv_dataframe, temp_dataframe])
    
    # Print the mean and standard deviation of the C-index results
    print(f"C-index mean CV:", np.mean(C_index_results_list))
    print(f"C-index standard deviation CV:", np.std(C_index_results_list))
    
    return C_index_results_list, cv_dataframe